# Lab 1: RDD Fundamentals - Solutions

**Objective**: Learn the basics of Resilient Distributed Datasets (RDDs) - Spark's fundamental data structure.

**Learning Outcomes**:
- Understand RDD properties: immutable, distributed, resilient
- Create RDDs from collections and data sources
- Apply basic transformations and actions
- Explore RDD lineage and fault tolerance

**Estimated Time**: 45 minutes

---

## Setup and Imports

First, let's set up our Spark environment and import necessary libraries.

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pandas as pd
import time

# Initialize Spark Session
spark = SparkSession.builder \
    .appName("Lab1-RDD-Fundamentals") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

# Get Spark Context
sc = spark.sparkContext

print(f"Spark version: {spark.version}")
ui_url = spark.sparkContext.uiWebUrl
print(f"Spark UI available at: {ui_url}")
print("💡 In GitHub Codespaces: Check the 'PORTS' tab below for forwarded port 4040 to access Spark UI")

## Part 1: Creating RDDs

RDDs can be created in several ways. Let's explore the most common methods.

### 1.1 Creating RDDs from Collections

In [ ]:
# Create a simple RDD from a Python list
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
numbers_rdd = sc.parallelize(numbers)

print(f"RDD created with {numbers_rdd.getNumPartitions()} partitions")
print(f"First few elements: {numbers_rdd.take(5)}")

**Exercise 1.1**: Create an RDD from a list of your favorite programming languages and specify 3 partitions.

*Hint: Use `sc.parallelize()` with the `numSlices` parameter*

In [ ]:
# Solution: Create languages_rdd from a list of programming languages with 3 partitions
languages = ["Python", "Scala", "Java", "R", "SQL", "JavaScript"]
languages_rdd = sc.parallelize(languages, numSlices=3)

# Validation
assert languages_rdd.getNumPartitions() == 3, "RDD should have 3 partitions"
assert len(languages_rdd.collect()) >= 3, "Should have at least 3 languages"
print("✓ Exercise 1.1 completed successfully!")
print(f"Languages RDD: {languages_rdd.collect()}")

### 1.2 Creating RDDs from Files

Let's load our synthetic customer data as an RDD.

In [ ]:
# Load customer data as text file (RDD of strings)
customers_text_rdd = sc.textFile("../Datasets/customers.csv")

print(f"Number of lines in customer file: {customers_text_rdd.count()}")
print(f"First line (header): {customers_text_rdd.first()}")

**Exercise 1.2**: Load the customer transactions file and skip the header line.

*Hint: Use `filter()` or slice the RDD*

In [ ]:
# Solution: Load transactions file and remove header
transactions_rdd = sc.textFile("../Datasets/customer_transactions.csv")
header = transactions_rdd.first()
transactions_no_header = transactions_rdd.filter(lambda line: line != header)

# Validation
assert transactions_no_header.count() > 0, "Should have transaction records"
assert header not in transactions_no_header.collect()[:10], "Header should be removed"
print("✓ Exercise 1.2 completed successfully!")
print(f"Transactions (without header): {transactions_no_header.count()} records")
print(f"Sample record: {transactions_no_header.take(1)[0]}")

## Part 2: Basic Transformations

Transformations are lazy operations that define a new RDD based on existing ones.

### 2.1 Map Transformation

In [ ]:
# Apply map transformation to square numbers
squared_rdd = numbers_rdd.map(lambda x: x ** 2)

print(f"Original: {numbers_rdd.collect()}")
print(f"Squared: {squared_rdd.collect()}")

**Exercise 2.1**: Parse the transaction CSV lines into structured data.

*Create an RDD where each element is a dictionary with keys: transaction_id, customer_id, amount, category*

In [ ]:
def parse_transaction(line):
    """Parse a CSV line into a transaction dictionary"""
    # Solution: Split the line and create a dictionary
    # Expected columns: transaction_id,customer_id,amount,category,payment_method,transaction_date,is_weekend,discount_applied
    fields = line.split(',')
    return {
        'transaction_id': fields[0],
        'customer_id': fields[1],
        'amount': float(fields[2]),
        'category': fields[3]
    }

# Solution: Apply the parsing function to create structured transaction RDD
structured_transactions = transactions_no_header.map(parse_transaction)

# Validation
sample_transaction = structured_transactions.first()
assert isinstance(sample_transaction, dict), "Should return dictionaries"
assert 'transaction_id' in sample_transaction, "Should have transaction_id key"
assert isinstance(sample_transaction['amount'], float), "Amount should be float"
print("✓ Exercise 2.1 completed successfully!")
print(f"Sample parsed transaction: {sample_transaction}")

### 2.2 Filter Transformation

In [ ]:
# Filter for even numbers
even_numbers = numbers_rdd.filter(lambda x: x % 2 == 0)

print(f"Even numbers: {even_numbers.collect()}")

**Exercise 2.2**: Filter transactions for high-value purchases (amount > $100).

In [ ]:
# Solution: Filter for transactions with amount greater than 100
high_value_transactions = structured_transactions.filter(lambda t: t['amount'] > 100)

# Validation
sample_high_value = high_value_transactions.take(5)
for transaction in sample_high_value:
    assert transaction['amount'] > 100, f"Amount {transaction['amount']} should be > 100"

print("✓ Exercise 2.2 completed successfully!")
print(f"High-value transactions: {high_value_transactions.count()} out of {structured_transactions.count()}")
print(f"Percentage: {(high_value_transactions.count() / structured_transactions.count()) * 100:.1f}%")

### 2.3 FlatMap Transformation

In [ ]:
# Example: Split words and flatten
sentences = ["Hello world", "Spark is awesome", "RDDs are cool"]
sentences_rdd = sc.parallelize(sentences)

# Map vs FlatMap comparison
words_map = sentences_rdd.map(lambda sentence: sentence.split(" "))
words_flatmap = sentences_rdd.flatMap(lambda sentence: sentence.split(" "))

print(f"Using map: {words_map.collect()}")
print(f"Using flatMap: {words_flatmap.collect()}")

**Exercise 2.3**: Extract all unique categories from transactions using flatMap.

In [ ]:
# Solution: Extract categories and get unique values
categories_rdd = structured_transactions.map(lambda t: t['category'])
unique_categories = categories_rdd.distinct()

# Validation
categories_list = unique_categories.collect()
assert len(categories_list) > 0, "Should have at least one category"
assert len(categories_list) == len(set(categories_list)), "Should be unique categories"
print("✓ Exercise 2.3 completed successfully!")
print(f"Unique categories: {sorted(categories_list)}")

## Part 3: Basic Actions

Actions trigger the execution of transformations and return results.

### 3.1 Collect and Count Actions

In [ ]:
# Demonstrate different actions
print(f"Count: {numbers_rdd.count()}")
print(f"First element: {numbers_rdd.first()}")
print(f"Take 3: {numbers_rdd.take(3)}")
print(f"All elements: {numbers_rdd.collect()}")

**Exercise 3.1**: Calculate basic statistics for transaction amounts.

In [ ]:
# Solution: Extract amounts and calculate statistics
amounts_rdd = structured_transactions.map(lambda t: t['amount'])

total_count = amounts_rdd.count()
total_amount = amounts_rdd.reduce(lambda a, b: a + b)
min_amount = amounts_rdd.min()
max_amount = amounts_rdd.max()
avg_amount = total_amount / total_count

# Validation
assert total_count > 0, "Should have transactions"
assert total_amount > 0, "Total amount should be positive"
assert min_amount <= avg_amount <= max_amount, "Statistics should be consistent"

print("✓ Exercise 3.1 completed successfully!")
print(f"Transaction Statistics:")
print(f"  Count: {total_count:,}")
print(f"  Total Amount: ${total_amount:,.2f}")
print(f"  Average Amount: ${avg_amount:.2f}")
print(f"  Min Amount: ${min_amount:.2f}")
print(f"  Max Amount: ${max_amount:.2f}")

### 3.2 Reduce Actions

In [ ]:
# Example: Find maximum number using reduce
maximum = numbers_rdd.reduce(lambda a, b: max(a, b))
sum_total = numbers_rdd.reduce(lambda a, b: a + b)

print(f"Maximum: {maximum}")
print(f"Sum: {sum_total}")

**Exercise 3.2**: Find the customer with the highest total spending.

In [ ]:
# Solution: Group by customer and find highest spender
# Step 1: Create (customer_id, amount) pairs
customer_amounts = structured_transactions.map(lambda t: (t['customer_id'], t['amount']))

# Step 2: Sum amounts by customer
customer_totals = customer_amounts.reduceByKey(lambda a, b: a + b)

# Step 3: Find customer with maximum total
top_customer = customer_totals.reduce(lambda a, b: a if a[1] > b[1] else b)

# Validation
assert isinstance(top_customer, tuple), "Should return a tuple (customer_id, amount)"
assert len(top_customer) == 2, "Should have customer_id and amount"
assert top_customer[1] > 0, "Top customer should have positive spending"

print("✓ Exercise 3.2 completed successfully!")
print(f"Top customer: {top_customer[0]} with total spending: ${top_customer[1]:.2f}")

## Part 4: RDD Lineage and Persistence

Understanding RDD lineage is crucial for fault tolerance and performance optimization.

### 4.1 Examining Lineage

In [ ]:
# Create a chain of transformations
pipeline_rdd = numbers_rdd \
    .filter(lambda x: x > 5) \
    .map(lambda x: x * 2) \
    .filter(lambda x: x < 20)

print(f"Final result: {pipeline_rdd.collect()}")
print(f"Lineage information:")
# Fixed: Use toDebugString() to show lineage instead of accessing dependencies directly
print(f"Debug string:\n{pipeline_rdd.toDebugString().decode('utf-8')}")
print(f"Number of partitions: {pipeline_rdd.getNumPartitions()}")

**Exercise 4.1**: Create a processing pipeline and examine its lineage.

In [ ]:
# Solution: Create a processing pipeline with multiple transformations
electronics_pipeline = structured_transactions \
    .filter(lambda t: t['category'] == 'Electronics') \
    .filter(lambda t: t['amount'] > 50) \
    .map(lambda t: t['amount'])

# Execute and examine lineage
result_count = electronics_pipeline.count()
sample_amounts = electronics_pipeline.take(5)

# Validation
assert result_count > 0, "Should have some electronics transactions > $50"
for amount in sample_amounts:
    assert amount > 50, f"Amount {amount} should be > $50"

print("✓ Exercise 4.1 completed successfully!")
print(f"Electronics transactions > $50: {result_count}")
print(f"Sample amounts: {sample_amounts}")
print(f"Pipeline lineage:\n{electronics_pipeline.toDebugString().decode('utf-8')}")

### 4.2 Persistence and Caching

In [ ]:
# Demonstrate caching benefits
import time

# Create expensive computation
expensive_rdd = structured_transactions.filter(lambda t: t['amount'] > 100).map(lambda t: t['amount'])

# Time without caching
start_time = time.time()
count1 = expensive_rdd.count()
sum1 = expensive_rdd.sum()
time_without_cache = time.time() - start_time

# Cache and time with caching
expensive_rdd.cache()
start_time = time.time()
count2 = expensive_rdd.count()  # This will cache the RDD
sum2 = expensive_rdd.sum()      # This will use the cache
time_with_cache = time.time() - start_time

print(f"Without cache: {time_without_cache:.3f} seconds")
print(f"With cache: {time_with_cache:.3f} seconds")
print(f"Results - Count: {count1}, Sum: {sum1:.2f}")

# Clean up cache
expensive_rdd.unpersist()

**Exercise 4.2**: Implement caching for a reusable computation.

In [ ]:
# Solution: Create an RDD that will be used multiple times and cache it
frequently_used_rdd = structured_transactions \
    .filter(lambda t: t['amount'] > 25) \
    .map(lambda t: (t['category'], t['amount']))

# Solution: Cache the RDD
frequently_used_rdd.cache()

# Perform multiple actions to benefit from caching
total_records = frequently_used_rdd.count()
categories = frequently_used_rdd.map(lambda x: x[0]).distinct().collect()
avg_by_category = frequently_used_rdd.groupByKey().mapValues(lambda amounts: sum(amounts) / len(amounts)).collect()

# Validation
assert total_records > 0, "Should have cached records"
assert len(categories) > 0, "Should have categories"
assert frequently_used_rdd.is_cached, "RDD should be cached"

print("✓ Exercise 4.2 completed successfully!")
print(f"Cached RDD statistics:")
print(f"  Total records: {total_records}")
print(f"  Categories: {categories}")
print(f"  Is cached: {frequently_used_rdd.is_cached}")

# Clean up
frequently_used_rdd.unpersist()

## Part 5: Working with Key-Value RDDs

Key-value RDDs enable powerful operations like joins and grouping.

### 5.1 Creating Key-Value RDDs

In [ ]:
# Create key-value RDD from transactions
category_amounts = structured_transactions.map(lambda t: (t['category'], t['amount']))

print(f"Sample key-value pairs: {category_amounts.take(5)}")
print(f"Keys only: {category_amounts.keys().distinct().collect()}")
print(f"Values only: {category_amounts.values().take(5)}")

**Exercise 5.1**: Analyze spending patterns by customer and category.

In [ ]:
# Solution: Create customer-category spending analysis
# Step 1: Create ((customer_id, category), amount) pairs
customer_category_pairs = structured_transactions.map(lambda t: 
    ((t['customer_id'], t['category']), t['amount'])
)

# Step 2: Sum amounts by customer-category combination
customer_category_totals = customer_category_pairs.reduceByKey(lambda a, b: a + b)

# Step 3: Find top spending customer-category combinations
top_combinations = customer_category_totals.sortBy(lambda x: x[1], ascending=False).take(10)

# Validation
assert len(top_combinations) > 0, "Should have customer-category combinations"
for combo in top_combinations[:3]:
    assert len(combo[0]) == 2, "Key should be (customer_id, category) tuple"
    assert combo[1] > 0, "Amount should be positive"

print("✓ Exercise 5.1 completed successfully!")
print("Top 10 Customer-Category Spending Combinations:")
for i, ((customer, category), amount) in enumerate(top_combinations, 1):
    print(f"  {i:2d}. {customer} - {category}: ${amount:.2f}")

### 5.2 GroupByKey and ReduceByKey

In [ ]:
# Compare groupByKey vs reduceByKey
from pyspark.sql.functions import col

# Using reduceByKey (more efficient)
category_totals_reduce = category_amounts.reduceByKey(lambda a, b: a + b)

# Using groupByKey (less efficient)
category_totals_group = category_amounts.groupByKey().mapValues(lambda amounts: sum(amounts))

print("Category totals using reduceByKey:")
for category, total in category_totals_reduce.collect():
    print(f"  {category}: ${total:.2f}")

# Verify they produce the same results (with rounding to handle floating point precision)
reduce_results = {k: round(v, 2) for k, v in category_totals_reduce.collect()}
group_results = {k: round(v, 2) for k, v in category_totals_group.collect()}
assert reduce_results == group_results, "Both methods should produce same results"

**Exercise 5.2**: Calculate average transaction amount by category using both methods.

In [ ]:
# Method 1: Using groupByKey
# Solution: Calculate average amounts by category using groupByKey
category_averages_group = category_amounts.groupByKey().mapValues(lambda amounts: 
    sum(amounts) / len(amounts)
)

# Method 2: Using reduceByKey (more efficient approach)
# Solution: Calculate average using sum and count
category_sums = category_amounts.reduceByKey(lambda a, b: a + b)
category_counts = category_amounts.mapValues(lambda x: 1).reduceByKey(lambda a, b: a + b)

# Join sums and counts to calculate averages
category_averages_reduce = category_sums.join(category_counts).mapValues(lambda x: x[0] / x[1])

# Collect results with rounding to handle floating point precision
averages_group = {k: round(v, 2) for k, v in category_averages_group.collect()}
averages_reduce = {k: round(v, 2) for k, v in category_averages_reduce.collect()}

# Validation
assert len(averages_group) > 0, "Should have category averages"
for category in averages_group:
    assert abs(averages_group[category] - averages_reduce[category]) < 0.01, "Both methods should produce similar results"

print("✓ Exercise 5.2 completed successfully!")
print("Average transaction amounts by category:")
for category, avg in sorted(averages_group.items()):
    print(f"  {category}: ${avg:.2f}")

## Summary and Cleanup

Congratulations! You've completed Lab 1 on RDD Fundamentals.

### Key Concepts Learned:

1. **RDD Creation**: From collections and files
2. **Transformations**: map, filter, flatMap (lazy evaluation)
3. **Actions**: collect, count, reduce (trigger execution)
4. **Lineage**: Understanding RDD dependencies and fault tolerance
5. **Persistence**: Caching for performance optimization
6. **Key-Value Operations**: reduceByKey, groupByKey, join

### Performance Tips:
- Use `reduceByKey` instead of `groupByKey` when possible
- Cache RDDs that will be used multiple times
- Minimize data shuffling operations
- Use appropriate partitioning for your data

In [ ]:
# Clean up Spark session
spark.stop()
print("Lab 1 completed successfully! 🎉")
print("Next: Lab 2 - Transformations vs Actions")